# Transient Storage

These opcodes behave almostidentically to storage but changes are discarded after every transaction.

#### Transient Storage Load

In [4]:
def tload(evm): 
    key = evm.stack.pop().value
    warm, value = evm.storage.load(key)
    evm.stack.push(value)

    evm.gas_dec(100)
    evm.pc += 1

#### Transient Storage Store

In [3]:
def tstore(evm): 
    key, value = evm.stack.pop(), evm.stack.pop()
    evm.storage.store(key, value)
    evm.gas_dec(100)
    evm.pc += 1